# CLEANING THE DATASET FOR TRAINING THE MODEL
## 1-Import useful modules

In [1]:
import pandas as pd
import numpy as np

## 2-Importing the dataset

In [2]:
#always display all the columns , so there is no need to specify each time the number of columns i want 
pd.set_option('display.max_columns', None)
# Chemin du fichier log
log_file_path = '../../data/raw/conn.log.labeled'
#charger les noms des columns
data_columns = pd.read_csv(log_file_path , sep='\t', skiprows=6, nrows=1, header=None).iloc[0][1:]
# Charger le fichier log en DataFrame
data_df = pd.read_csv(log_file_path, sep='\t', comment='#',header=None)
#specify the labels of the columns 
data_df.columns = data_columns

In [13]:
# Check dataset shape
data_df.shape #(23145, 21)
# Check dataset head
data_df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents label detailed-label
0,1.545404e+09,CrDn63WjJEmrWGjqf,192.168.1.195,41040,185.244.25.235,80,tcp,-,3.139211,0,0,S0,-,-,0,S,3,180,0,0,- Benign -
1,1.545404e+09,CY9lJW3gh1Eje4usP6,192.168.1.195,41040,185.244.25.235,80,tcp,-,-,-,-,S0,-,-,0,S,1,60,0,0,- Benign -
2,1.545404e+09,CcFXLynukEDnUlvgl,192.168.1.195,41040,185.244.25.235,80,tcp,-,-,-,-,S0,-,-,0,S,1,60,0,0,- Benign -
3,1.545404e+09,CDrkrSobGYxHhYfth,192.168.1.195,41040,185.244.25.235,80,tcp,http,1.477656,149,128252,SF,-,-,2896,ShADadttcfF,94,5525,96,139044,- Benign -
4,1.545404e+09,CTWZQf2oJSvq6zmPAc,192.168.1.195,41042,185.244.25.235,80,tcp,-,3.147116,0,0,S0,-,-,0,S,3,180,0,0,- Benign -


we notice : 
- some fields with '-', which means the field is unset according to the dataset documentation.
- the last column correspond to three separate columns ! ( we need more lines to notice this n you can check the dataset directly to see this problem or just keep going and you will understand in the part 3-cleaning the dataset)

In [4]:
# Check dataset info
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23145 entries, 0 to 23144
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ts                                       23145 non-null  float64
 1   uid                                      23145 non-null  object 
 2   id.orig_h                                23145 non-null  object 
 3   id.orig_p                                23145 non-null  int64  
 4   id.resp_h                                23145 non-null  object 
 5   id.resp_p                                23145 non-null  int64  
 6   proto                                    23145 non-null  object 
 7   service                                  23145 non-null  object 
 8   duration                                 23145 non-null  object 
 9   orig_bytes                               23145 non-null  object 
 10  resp_bytes                               23145

- So here there are no null cells but we have cells with the value "-" .
- Some numerical fields are misidentified as "object" (strings) which must be fixed later.

## 3-Cleaning the dataset

In [5]:
# Split the last combined column into three ones
tunnel_parents_column = data_df.iloc[:,-1].apply(lambda x: x.split()[0])
label_column = data_df.iloc[:,-1].apply(lambda x: x.split()[1])
label_detailed_column = data_df.iloc[:,-1].apply(lambda x: x.split()[2])
#we didn't yet change the dataframe ,now we just have a three separate columns 

In [6]:
# Remove the combined column
data_df.drop(["tunnel_parents   label   detailed-label"], axis=1, inplace=True)
data_df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
0,1.545404e+09,CrDn63WjJEmrWGjqf,192.168.1.195,41040,185.244.25.235,80,tcp,-,3.139211,0,0,S0,-,-,0,S,3,180,0,0
1,1.545404e+09,CY9lJW3gh1Eje4usP6,192.168.1.195,41040,185.244.25.235,80,tcp,-,-,-,-,S0,-,-,0,S,1,60,0,0
2,1.545404e+09,CcFXLynukEDnUlvgl,192.168.1.195,41040,185.244.25.235,80,tcp,-,-,-,-,S0,-,-,0,S,1,60,0,0
3,1.545404e+09,CDrkrSobGYxHhYfth,192.168.1.195,41040,185.244.25.235,80,tcp,http,1.477656,149,128252,SF,-,-,2896,ShADadttcfF,94,5525,96,139044
4,1.545404e+09,CTWZQf2oJSvq6zmPAc,192.168.1.195,41042,185.244.25.235,80,tcp,-,3.147116,0,0,S0,-,-,0,S,3,180,0,0


In [7]:
# Add newly created columns to the dataset
data_df["tunnel_parents"] = tunnel_parents_column
data_df["label"] = label_column
data_df["detailed_label"] = label_detailed_column
data_df.head()
# NOW we are so goooood

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed_label
0,1.545404e+09,CrDn63WjJEmrWGjqf,192.168.1.195,41040,185.244.25.235,80,tcp,-,3.139211,0,0,S0,-,-,0,S,3,180,0,0,-,Benign,-
1,1.545404e+09,CY9lJW3gh1Eje4usP6,192.168.1.195,41040,185.244.25.235,80,tcp,-,-,-,-,S0,-,-,0,S,1,60,0,0,-,Benign,-
2,1.545404e+09,CcFXLynukEDnUlvgl,192.168.1.195,41040,185.244.25.235,80,tcp,-,-,-,-,S0,-,-,0,S,1,60,0,0,-,Benign,-
3,1.545404e+09,CDrkrSobGYxHhYfth,192.168.1.195,41040,185.244.25.235,80,tcp,http,1.477656,149,128252,SF,-,-,2896,ShADadttcfF,94,5525,96,139044,-,Benign,-
4,1.545404e+09,CTWZQf2oJSvq6zmPAc,192.168.1.195,41042,185.244.25.235,80,tcp,-,3.147116,0,0,S0,-,-,0,S,3,180,0,0,-,Benign,-


### NOW we remove useless columns 
#### Let's remove the columns with a unique value or calumns with unique values for each log : zero added information for our model

In [8]:
# Check the number of unique values in each column
data_df.nunique().sort_values(ascending=False)

0
ts                23145
uid               23145
duration           4654
id.orig_p          4383
orig_ip_bytes       108
resp_ip_bytes        62
orig_pkts            53
id.resp_h            49
resp_bytes           44
orig_bytes           29
resp_pkts            28
history              26
id.resp_p            10
conn_state            6
service               5
detailed_label        4
missed_bytes          3
proto                 2
id.orig_h             2
label                 2
local_orig            1
local_resp            1
tunnel_parents        1
dtype: int64

In [9]:
# Two columns have only unique values and three columns have only one value, so we should drop them.
data_df.drop(columns=["ts","uid","local_resp","local_orig","tunnel_parents"], inplace=True)


Now let's do some extra cleaning 

In [10]:
# IP addresses might introduce bias since we don't care which machine we are trying to deffend . 
data_df.drop(columns=["id.orig_h","id.resp_h"], inplace=True)

# We just need the label of the log ,so the "detailed_label" doesn't contribute to the data analysis.
data_df.drop(columns="detailed_label", inplace=True)

# Check the dataset
data_df.head()

,id.orig_p,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,41040,80,tcp,-,3.139211,0,0,S0,0,S,3,180,0,0,Benign
1,41040,80,tcp,-,-,-,-,S0,0,S,1,60,0,0,Benign
2,41040,80,tcp,-,-,-,-,S0,0,S,1,60,0,0,Benign
3,41040,80,tcp,http,1.477656,149,128252,SF,2896,ShADadttcfF,94,5525,96,139044,Benign
4,41042,80,tcp,-,3.147116,0,0,S0,0,S,3,180,0,0,Benign


### Fix unset values and data types

In [11]:
# Replace all occurrences of empty/unset cells with null values
data_df.replace({'-':np.nan, "(empty)":np.nan}, inplace=True)
# Check the dataset
data_df.head()

,id.orig_p,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,41040,80,tcp,NaN,3.139211,0,0,S0,0,S,3,180,0,0,Benign
1,41040,80,tcp,NaN,NaN,NaN,NaN,S0,0,S,1,60,0,0,Benign
2,41040,80,tcp,NaN,NaN,NaN,NaN,S0,0,S,1,60,0,0,Benign
3,41040,80,tcp,http,1.477656,149,128252,SF,2896,ShADadttcfF,94,5525,96,139044,Benign
4,41042,80,tcp,NaN,3.147116,0,0,S0,0,S,3,180,0,0,Benign


In [12]:
# Fix data types of the misinterpreted columns
dtype_convert_dict = {
    "duration": float,
    "orig_bytes": float,
    "resp_bytes": float
}
data_df = data_df.astype(dtype_convert_dict)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23145 entries, 0 to 23144
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id.orig_p      23145 non-null  int64  
 1   id.resp_p      23145 non-null  int64  
 2   proto          23145 non-null  object 
 3   service        1847 non-null   object 
 4   duration       5321 non-null   float64
 5   orig_bytes     5321 non-null   float64
 6   resp_bytes     5321 non-null   float64
 7   conn_state     23145 non-null  object 
 8   missed_bytes   23145 non-null  int64  
 9   history        23145 non-null  object 
 10  orig_pkts      23145 non-null  int64  
 11  orig_ip_bytes  23145 non-null  int64  
 12  resp_pkts      23145 non-null  int64  
 13  resp_ip_bytes  23145 non-null  int64  
 14  label          23145 non-null  object 
dtypes: float64(3), int64(7), object(5)
memory usage: 2.6+ MB


## 4. STORING CLEANED DATASET

In [13]:
# Store cleaned dataset to a csv file
data_df.to_csv("../../data/interim/conn_log_labeled-cleaned.csv")
data_df.head()

,id.orig_p,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,41040,80,tcp,NaN,3.139211,0.0,0.0,S0,0,S,3,180,0,0,Benign
1,41040,80,tcp,NaN,NaN,NaN,NaN,S0,0,S,1,60,0,0,Benign
2,41040,80,tcp,NaN,NaN,NaN,NaN,S0,0,S,1,60,0,0,Benign
3,41040,80,tcp,http,1.477656,149.0,128252.0,SF,2896,ShADadttcfF,94,5525,96,139044,Benign
4,41042,80,tcp,NaN,3.147116,0.0,0.0,S0,0,S,3,180,0,0,Benign
